# K - Nearest Neighbours Implementation (KNN)

This notebook explains various NumPy techniques that can be used to implement the KNN algorithm from scratch.

## About K - Nearest Neighbours

K - Nearest Neighbours is a non-parameteric machine learning algorithm which can be used for both classification and regression. Here $k$ is the hyper parameter for the model. This model is usually evaluated with different values of $k$ to find the best value of the $k$. It uses distance measuring function (which computes the similarities between the points). Typically a squared euclidean distance is used.
### Algorithm:
#### Training:
Since KNN uses the raw data points for prediction, there is no model training occurs. we just simply store the Training points as it is.
#### Classification:
1. Calculate the distance between test point $\mathbf{\hat x}$ and all the points $\mathbf{x_i}$ in the training set.
2. Argsort the distances and take the first k values inorder to find the indices of the K nearest neighbours from the training set.
3. Predict the class of the test point $\hat y$ by majority voting(i.e, the class with most occurences), using the classes $y_i$ of the k nearest neighbours.

#### Regression:
1. Step 1 and 2 same as for classification.
2. Predict the value of $\hat y$ by computing the mean of the $y_i$ of the k nearest neighbours.

## Importing Modules

Lets import all the required modules. This notebook only uses NumPy for the KNN algorithm. Pandas is just used for loading  the dataset.

In [1]:
import numpy as np
import pandas as pd

## Loading and processing the Diabetes dataset

Now lets load the pima indians diabetes dataset using Pandas. As the dataset is loaded as pd.DataFrame, we use variable `diabetes_df` for stroing the dataset. 

**Note:** The suffix "df"(short for DataFrame) is used with pd.DataFrame objects to differentiate them.

In [35]:
diabetes_df = pd.read_csv("datasets/pima-indians-diabetes.csv")
diabetes_df.head()

,1. Number of times pregnant,2. Plasma glucose concentration a 2 hours in an oral glucose tolerance test,3. Diastolic blood pressure (mm Hg),4. Triceps skin fold thickness (mm),5. 2-Hour serum insulin (mu U/ml),6. Body mass index (weight in kg/(height in m)^2),7. Diabetes pedigree function,8. Age (years),9. Class variable (0 or 1)
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


### Spliting Training and Testing data

In [36]:
X = diabetes_df.iloc[:,:-1].values
y = diabetes_df.iloc[:,-1].values

### Min - Max Normalization (Min - Max scaling)

In [37]:
X_min, X_max = X.min(axis = 0) , X.max(axis =0)
X = X_min + X/(X_max-X_min)
X 

array([[ 0.35294118,  0.74371859,  0.59016393, ...,  0.50074516,
         0.3457199 , 21.83333333],
       [ 0.05882353,  0.42713568,  0.54098361, ...,  0.39642325,
         0.2278719 , 21.51666667],
       [ 0.47058824,  0.91959799,  0.52459016, ...,  0.34724292,
         0.36493424, 21.53333333],
       ...,
       [ 0.29411765,  0.6080402 ,  0.59016393, ...,  0.390462  ,
         0.18261144, 21.5       ],
       [ 0.05882353,  0.63316583,  0.49180328, ...,  0.4485842 ,
         0.22701793, 21.78333333],
       [ 0.05882353,  0.46733668,  0.57377049, ...,  0.45305514,
         0.21250043, 21.38333333]])

Now lets split the data set into training and testing sets with a split ratio of 7:3.

In [38]:
split_ratio = .7
sl = int(X.shape[0]*split_ratio)
X_train,y_train = X[:sl],y[:sl]
X_test,y_test = X[sl:],y[sl:]

## Calculating Squared euclidean distance

The squared euclidean distance between two feature vectors (or data points) $\mathbf{a} = (a_1,a_2,...,a_n)^T$ and $\mathbf{b} = (b_1,b_2,...,b_n)^T$ can be computed as follows

$$
\text{sqEuclidean}(\mathbf{a},\mathbf{b}) 
= \sum_{i=0}^{n} (a_i - b_i)^2 
= (\mathbf{a}- \mathbf{b}) \cdot (\mathbf{a} - \mathbf{b}) 
= (\mathbf{a} - \mathbf{b})^T (\mathbf{a} - \mathbf{b})
$$
### One vs One
Lets discuss multiple methods the squrared euclidean function can be implemented with numpy.

In [6]:
def sq_euclidean_1(a,b):
    """Squared Euclidean distance using summation."""
    return np.sum((a-b)**2)

def sq_euclidean_2(a,b):
    """Squrared Euclidean distance using dot product"""
    diff = a-b
    return np.dot(diff,diff)

def sq_euclidean_3(a,b):
    """Squared Euclidean distance using matrix multiplication"""
    diff = a-b
    return diff.T @ diff

In [7]:
a, b = X_train[0],X_train[1]
print(sq_euclidean_1(a,b))
print(sq_euclidean_2(a,b))
print(sq_euclidean_3(a,b))

0.3178707190193262
0.3178707190193262
0.3178707190193262


### Many vs One
Now lets try to find the squared euclidean distance between a set of $m$ points $A$ and a point $b$


In [8]:
A, b = X_train[:5],X_train[5]
print(sq_euclidean_1(A,b))
print(sq_euclidean_1(A,b)) 
print(sq_euclidean_1(A,b)) 

2.054106022293314
2.054106022293314
2.054106022293314


This is not the required result, we need $m = 5$ distances, thus the requried result must have the shape `(m,)`, in this case 5. 

So lets make changes to the code so that it could find m different distances.

In [9]:
def sq_euclidean_4(A,b):
    """Squared Euclidean distance using summation. Also supports A to be 2D"""
    return np.sum((A-b)**2, axis =-1) #Here -1 denotes the last axis

we cannot use dot product or matrix multiplication directly in this case because those funcions does not provide flexibility in specifying the axes on which the sum of products is calculated. Thus we could use `np.einsum` function to implement this.

In [10]:
def sq_euclidean_5(A,b):
    """Squared Euclidean distance using einstein summation. Also supports A to be 2D"""
    diff = A-b # numpy broad casting is used here
    return np.einsum("ij,ij->i",diff,diff) 

In [11]:
A, b = X_train[:5],X_train[5]
print(sq_euclidean_4(A,b))
print(sq_euclidean_5(A,b)) 

[0.31298594 0.17433988 0.19394755 0.16849053 1.20434212]
[0.31298594 0.17433988 0.19394755 0.16849053 1.20434212]


Thus we get our expected results. 

### Many vs Many
Now lets compute the distances between set of $m$ points $A$ and set of $n$ points $B$. Thus we need $m\times n$ distances and the expected shape is `(n,m)`. So to get the result as expected we make use of the array broadcasting feature in numpy.

**Note:** Here `(n,m)` is used instead of `(m,n)` because it is assumed that B will be the set of test points. thus it is convenient to have it in the 0-th axis.

In [12]:
def sq_euclidean_6(A,B):
    """Squared Euclidean distance using summation. Also supports A and B to be 2D"""
    return np.sum((A-B[:,np.newaxis,:])**2, axis =-1) 

def sq_euclidean_7(A,B):
    """Squared Euclidean distance using einstein summation. Also supports A and B to be 2D"""
    diff = A-B[:,np.newaxis,:] 
    return np.einsum("ijk,ijk->ij",diff,diff) # also added an additional dimension

**Note:** The only difference between the previous set and this was the introuction of a new axis to allow numpy broadcating.
Refer [this link](https://numpy.org/devdocs/user/theory.broadcasting.html) to know more about numpy array broadcasting rules

In [13]:
A, B = X_train[:3],X_train[3:7]
dist6 = sq_euclidean_6(A,B)
dist7 = sq_euclidean_7(A,B)
print(dist6)
print(dist7) 
print(f"Shape of A = {A.shape} and Shape of B = {B.shape}")
print("Is dist6 and dist7 are same?",np.all(dist6 == dist7))
print(f"Shape of dist6 = {dist6.shape} and shape of dist7 = {dist7.shape}")

[[0.4827717  0.05087282 0.54448802]
 [0.839176   0.90589191 1.04147037]
 [0.31298594 0.17433988 0.19394755]
 [0.38682202 0.05719462 0.54931948]]
[[0.4827717  0.05087282 0.54448802]
 [0.839176   0.90589191 1.04147037]
 [0.31298594 0.17433988 0.19394755]
 [0.38682202 0.05719462 0.54931948]]
Shape of A = (3, 8) and Shape of B = (4, 8)
Is dist6 and dist7 are same? True
Shape of dist6 = (4, 3) and shape of dist7 = (4, 3)


Now that we made our function accept two two dimensioal arrays, Lets generalize this so that it accepts $B$ in $N$ dimensions.

In [14]:
def sq_euclidean_8(A,B):
    """Squared Euclidean distance using summation. Supports A to be 2D and B to be N dimensional."""
    return np.sum((A-B[...,np.newaxis,:])**2, axis =-1) # added ellipsis to support N dimensions.

def sq_euclidean_9(A,B):
    """Squared Euclidean distance using einstein summation. Supports A to be 2D and B to be N dimensional."""
    diff = A-B[...,np.newaxis,:] 
    return np.einsum("...i,...i->...",diff,diff) # added ellipis to support N dimensions.

In [15]:
A, B = X_train[:3],X_train[3:9].reshape(2,3,-1)
dist6 = sq_euclidean_8(A,B)
dist7 = sq_euclidean_9(A,B)
print(dist6)
print(dist7) 
print(f"Shape of A = {A.shape} and Shape of B = {B.shape}")
print("Is dist6 and dist7 are same?",np.all(dist6 == dist7))
print(f"Shape of dist6 = {dist6.shape} and shape of dist7 = {dist7.shape}")

[[[0.4827717  0.05087282 0.54448802]
  [0.839176   0.90589191 1.04147037]
  [0.31298594 0.17433988 0.19394755]]

 [[0.38682202 0.05719462 0.54931948]
  [0.72359709 0.70798156 0.49305372]
  [0.58316553 0.90399208 0.93269195]]]
[[[0.4827717  0.05087282 0.54448802]
  [0.839176   0.90589191 1.04147037]
  [0.31298594 0.17433988 0.19394755]]

 [[0.38682202 0.05719462 0.54931948]
  [0.72359709 0.70798156 0.49305372]
  [0.58316553 0.90399208 0.93269195]]]
Shape of A = (3, 8) and Shape of B = (2, 3, 8)
Is dist6 and dist7 are same? True
Shape of dist6 = (2, 3, 3) and shape of dist7 = (2, 3, 3)


### Squared Euclidean distance using cdist
`cdist` is a function defined in the scipy library, which can be used to calculate the distances using various metrics including squared euclidean distance. But `cdist` require $A$ and $B$ to be strictly 2 dimensional.

In [16]:
from scipy.spatial.distance import cdist

In [17]:
A, B = X_train[:3],X_train[3:9]
dist = cdist(B,A,metric="sqeuclidean") # B, A is used instead of A, B, to be consistent with out previous implementations.
print(dist)
print("Shape of dist is :", dist.shape)

[[0.4827717  0.05087282 0.54448802]
 [0.839176   0.90589191 1.04147037]
 [0.31298594 0.17433988 0.19394755]
 [0.38682202 0.05719462 0.54931948]
 [0.72359709 0.70798156 0.49305372]
 [0.58316553 0.90399208 0.93269195]]
Shape of dist is : (6, 3)


In [18]:
sq_euclidean_9(A,B) == cdist(B,A,metric="sqeuclidean")

array([[ True,  True,  True],
       [ True, False,  True],
       [ True, False,  True],
       [False,  True, False],
       [ True, False, False],
       [ True,  True,  True]])

### Comparision of implementations

In [19]:
%%timeit
sq_euclidean_8(A,B)

22.9 µs ± 2.05 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [20]:
%%timeit
sq_euclidean_9(A,B)

12.3 µs ± 131 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [21]:
%%timeit
cdist(B,A,metric="sqeuclidean")

28 µs ± 1.46 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


From the above implementations it is clear that the `sq_euclidean_9` (using np.einsum) perfroms the fastest.

## Finding the y Values of the k nearest neignbours

Now lets find the y values of the k nearest neighbours.
To do that 
1. Use the `np.argsort` to get the indices in order of distances in acending order. 
2. Slice the first k indices to get the indices of the k nearest neignbours
3. by using indexing on y with the computed indices, we get the required y values

In [22]:
k = 3
k_nearest_ys = y_train[np.argsort(sq_euclidean_9(X_train,X_test))[...,:k]]
print(
    k_nearest_ys[0],
    k_nearest_ys[3],
    k_nearest_ys[5],
    sep="\n"
)

[1 0 0]
[0 0 0]
[1 0 0]


##### Majority votimg for calssification
1. Find the classes available in the training using `np.unique` and store it in `classes`
2. Compute the equality tensor for each point by equating it with the classes in a fashion similar to outer product.
3. Compute the sum of the equality tensor to get the comut tensor, containing the count of each classes among the nearest neighbours.
4. Using the `np.argmax` find the index of the class with the maximum count among the nearest neighbours.
5. Get the class name from at the found index by indexing the `classes`


In [23]:
classes = np.unique(y_train)
equality_tensor = k_nearest_ys[...,np.newaxis,:] == classes.reshape(-1,1)
count_tensor = np.sum(equality_tensor,axis = -1)
max_count_class_index = np.argmax(count_tensor, axis = -1)
y_hat = classes[max_count_class_index]
print(
    "Nearest ys of 0, 3 and 5",
    k_nearest_ys[0],
    k_nearest_ys[3],
    k_nearest_ys[5],
    sep="\n"
)
print(
    "Equality tensor of 0, 3 and 5",
    equality_tensor[0],
    equality_tensor[3],
    equality_tensor[5],
    sep="\n"
)
print(
    "Count tensor of 0, 3 and 5",
    count_tensor[0],
    count_tensor[3],
    count_tensor[5],
    sep="\n"
)
print(
    "Max count class index of 0, 3 and 5",
    max_count_class_index[0],
    max_count_class_index[3],
    max_count_class_index[5],
    sep="\n"
)
print(
    "Predicted class of 0, 3 and 5",
    y_hat[0],
    y_hat[3],
    y_hat[5],
    sep="\n"
)

Nearest ys of 0, 3 and 5
[1 0 0]
[0 0 0]
[1 0 0]
Equality tensor of 0, 3 and 5
[[False  True  True]
 [ True False False]]
[[ True  True  True]
 [False False False]]
[[False  True  True]
 [ True False False]]
Count tensor of 0, 3 and 5
[2 1]
[3 0]
[2 1]
Max count class index of 0, 3 and 5
0
0
0
Predicted class of 0, 3 and 5
0
0
0


## Class for KNN Classifier

Lets Ecapsulate everything we discussed into a python class

In [24]:
class KNN:
    '''Base class for KNN'''
    def __init__(self,k=3,dist=sq_euclidean_9):
        self.dist = dist
        self.k = k
    
    def fit(self,X_train,y_train):
        self.X_train = X_train
        self.y_train = y_train
        
    def k_nearest_ys(self,X_test):
        '''Computes the y values of the k nearest neighbours.\ 
This is computed for both classification and regression.'''
        return self.y_train[np.argsort(self.dist(self.X_train,X_test))[...,:self.k]]
    
class KNNClassifier(KNN):
    """K Nearest Neighbour Classifier"""
    def fit(self,X_train,y_train):
        super().fit(X_train, y_train)
        self.classes = np.unique(y_train)
    
    def predict(self,X_test):
        '''Returns the predicted class for the points in test_x, i.e, classification result'''
        return self.classes[
            np.argmax(
                np.sum(
                    self.k_nearest_ys(X_test)[...,np.newaxis,:] == self.classes.reshape(-1,1),
                    axis = -1
                ),
                axis=-1
            )
        ]        

## Implementing the classifier

In [25]:
diabetes_predictor = KNNClassifier(k = 3)
diabetes_predictor.fit(X_train,y_train)
y_pred = diabetes_predictor.predict(X_test)

### Evaluating the Classifier model model

In [28]:
print(f"Acuracy of the classifier is :{(y_pred == y_test).mean()*100:.3f}",)

Acuracy of the classifier is :76.623


### Evaluating with multiple values for k

In [34]:
for k in range(4,7):
    diabetes_predictor = KNNClassifier(k = k)
    diabetes_predictor.fit(X_train,y_train)
    y_pred = diabetes_predictor.predict(X_test)
    print(f"Acuracy of the classifier with k = {k} is :{(y_pred == y_test).mean()*100:.3f}",)

Acuracy of the classifier with k = 4 is :0.800
Acuracy of the classifier with k = 5 is :0.000
Acuracy of the classifier with k = 6 is :0.000


## Class For KNN Regressor

In [ ]:
class KNNRegressor(KNN):
    '''K Nearest Neighbour Regressor'''
    def predict(self, X_test):
        '''Returns the predicted y values for the points in test_x, i.e, regression result'''
        return np.mean(self.k_nearest_ys(X_test), axis = -1)

## Loading and processing Real estate Dataset 

In [43]:
real_estate_df = pd.read_csv("datasets/real-estate.csv")
real_estate_df.head()

,No,X1 transaction date,X2 house age,X3 distance to the nearest MRT station,X4 number of convenience stores,X5 latitude,X6 longitude,Y house price of unit area
0,1,2012.917,32.0,84.87882,10,24.98298,121.54024,37.9
1,2,2012.917,19.5,306.59470,9,24.98034,121.53951,42.2
2,3,2013.583,13.3,561.98450,5,24.98746,121.54391,47.3
3,4,2013.500,13.3,561.98450,5,24.98746,121.54391,54.8
4,5,2012.833,5.0,390.56840,5,24.97937,121.54245,43.1


### Spliting Training and Testing data

Here we also get rid of the first column (No) since it does not help in regression

In [44]:
X = real_estate_df.iloc[:,1:-1].values 
y = real_estate_df.iloc[:,-1].values

### Min - Max Normalization (Min - Max scaling)

In [45]:
X_min, X_max = X.min(axis = 0) , X.max(axis =0)
X = X_min + X/(X_max-X_min)
X 

array([[4.21017464e+03, 7.30593607e-01, 2.33959697e+01, 1.00000000e+00,
        3.27682676e+02, 1.43202173e+03],
       [4.21017464e+03, 4.45205479e-01, 2.34302664e+01, 9.00000000e-01,
        3.27650684e+02, 1.43201386e+03],
       [4.21090172e+03, 3.03652968e-01, 2.34697721e+01, 5.00000000e-01,
        3.27736966e+02, 1.43206130e+03],
       ...,
       [4.21053818e+03, 4.29223744e-01, 2.34433182e+01, 7.00000000e-01,
        3.27637232e+02, 1.43201763e+03],
       [4.21026525e+03, 1.84931507e-01, 2.33990528e+01, 5.00000000e-01,
        3.27485875e+02, 1.43202637e+03],
       [4.21081110e+03, 1.48401826e-01, 2.33968324e+01, 9.00000000e-01,
        3.27577853e+02, 1.43205257e+03]])

Now lets split the data set into training and testing sets with a split ratio of 7:3.

In [46]:
split_ratio = .7
sl = int(X.shape[0]*split_ratio)
X_train,y_train = X[:sl],y[:sl]
X_test,y_test = X[sl:],y[sl:]

## Implementing the KNN Regressor

In [47]:
housing_price_predictor = KNNRegressor(k = 3)
housing_price_predictor.fit(X_train,y_train)
y_pred = housing_price_predictor.predict(X_test)

### Evaluating KNN Regressor

In [50]:
rmse = np.sqrt(np.mean((y_test-y_pred)**2))
print(f"Root Mean Squared Error {rmse:.3f}")

Root Mean Squared Error 8.065
